In [129]:
from sec_api import QueryApi
import pandas as pd
import json
import pprint

In [203]:
# function for querying the api

def query_api(key=str, form_type=str, filed_at=str, ticker=str):
    queryApi = QueryApi(api_key=key)
    query = {
            "query": { "query_string": {
                "query": "formType:" + form_type + " AND filedAt:" + filed_at + " AND ticker:" + ticker
                } },
            "from": "0",
            "size": "100",
            "sort": [{ "filedAt": { "order": "desc" } }]
            }
    filings = queryApi.get_filings(query)

    output = filings['filings']

    return output

In [204]:
query_api("a623cd29617f549e2278cb906046e4136f6edd7e861cd072188a05569260f6bb", "11-K", "{2020-01-31 TO *}", '[\"\" TO *]')

[{'id': '2612ec3b7ae3c259b2ee5ef06f37c8c9',
  'accessionNo': '0001493152-23-026972',
  'cik': '1701756',
  'ticker': 'SDOT',
  'companyName': 'Sadot Group Inc.',
  'companyNameLong': 'Sadot Group Inc. (Filer)',
  'formType': '8-K',
  'description': 'Form 8-K - Current report - Item 2.02 Item 9.01',
  'filedAt': '2023-08-07T09:30:10-04:00',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1701756/000149315223026972/0001493152-23-026972.txt',
  'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1701756/000149315223026972/0001493152-23-026972-index.htm',
  'linkToXbrl': '',
  'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1701756/000149315223026972/form8-k.htm',
  'entities': [{'companyName': 'Sadot Group Inc. (Filer)',
    'cik': '1701756',
    'irsNo': '472555533',
    'stateOfIncorporation': 'NV',
    'fiscalYearEnd': '1231',
    'type': '8-K',
    'act': '34',
    'fileNo': '001-39223',
    'filmNo': '231145907',
    'sic': '5810 Retail-Eating &amp; Drinkin

In [215]:
queryApi = QueryApi(api_key="2ab0f49240e0aa82076149abf6d413bd9ab7c2ddf9db25c3d36379f31a590522")
query = {
  "query": { "query_string": {
      "query": "formType:\"11-K\" AND filedAt:{2020-01-31 TO *} AND ticker:[\"\" TO *]"
    } },
  "from": "0",
  "size": "100",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

KeyboardInterrupt: 

In [206]:
# print(filings)

output = filings['filings']

pprint(json.loads(output))

TypeError: the JSON object must be str, bytes or bytearray, not list

In [32]:
import json
with open('result.json', 'w') as fp:
    json.dump(filings, fp)

In [205]:
#matching function
from fuzzywuzzy import fuzz
def fuzzy_match(keyword, string):
    return fuzz.partial_ratio(keyword.lower(), string.lower()) >= 54

In [210]:
# function tidied version
# retrieving the matches from the F-11k filings 

output_list = []

for x in output:
    filing_to_search = x['entities']
    for dict in filing_to_search:
        sic = str(dict['sic'])
        ticker = x['ticker']
        company_name = x['companyName']
        cik = dict['cik']

        key_words = ["healthcare", "financial services", "target return funds", "balanced funds", "mid cap funds"]
        # target_string = "8090 Services-Misc finance services, Health & Allied care Services, NEC balance fund"

        matched_words = [word for word in key_words if fuzzy_match(word, sic)]

        output_dict={}
        if len(matched_words)>0:
            output_dict['ticker']=str(ticker)
            output_dict['company_name']=company_name
            output_dict['cik']=cik
            output_dict['matches']=matched_words
            output_list.append(output_dict)

# print(output_list)

# searching for the tickers with matches the Form-4

form_4_dict = {}


for match in output_list:
    ticker = f'{match["ticker"]}'
    if ticker not in form_4_dict.keys():
        # print('true')
        query_api("a623cd29617f549e2278cb906046e4136f6edd7e861cd072188a05569260f6bb", "4", "{2021-01-31 TO *}", ticker)
        
        for b in output:
            comp_name = (b['companyName'])
            form_4_comp_name_list = []
            if comp_name not in form_4_comp_name_list:
                form_4_comp_name_list.append(comp_name)
                print(form_4_comp_name_list)
                form_4_dict[ticker]=form_4_comp_name_list
            # output_dict['form_4']=form_4_comp_name_list
            
            # output_list.append(output_dict)

    # if ticker in form_4_dict.keys():
        

['HOME BANCSHARES INC']
['XCEL ENERGY INC']
['REVVITY, INC.']
['FIRST BANCORP /PR/']
['CONSUMER PORTFOLIO SERVICES, INC.']
['Mativ Holdings, Inc.']
['Mativ Holdings, Inc.']
['MAGNA INTERNATIONAL INC']
['Sleep Number Corp']
['AZZ INC']
['KROGER CO']
['KROGER CO']
['UWHARRIE CAPITAL CORP']
['PARKE BANCORP, INC.']
['Fortune Brands Innovations, Inc.']
['PROVIDENT FINANCIAL SERVICES INC']
['UWHARRIE CAPITAL CORP']
['PACWEST BANCORP']
['OLD SECOND BANCORP INC']
['MALVERN BANCORP, INC.']
['PARKE BANCORP, INC.']
['DYNEX CAPITAL INC']
['PARKE BANCORP, INC.']
['BCB BANCORP INC']
['Silvergate Capital Corp']
['Core Laboratories Inc. /DE/']
['DIAGEO PLC']
['Orange County Bancorp, Inc. /DE/']
['UNITED SECURITY BANCSHARES']
['DIAGEO PLC']
['VIAD CORP']
['LINCOLN NATIONAL CORP']
['LINCOLN NATIONAL CORP']
['ENZO BIOCHEM INC']
['BERKSHIRE HILLS BANCORP INC']
['POPULAR, INC.']
['Pathfinder Bancorp, Inc.']
['POPULAR, INC.']
['PBF Energy Inc.']
['SEACOAST BANKING CORP OF FLORIDA']
['Trane Technologies plc'

In [209]:
# retrieving the matches from the F-11k filings 

output_list = []

for x in output:
    filing_to_search = x['entities']
    for dict in filing_to_search:
        sic = str(dict['sic'])
        ticker = x['ticker']
        company_name = x['companyName']
        cik = dict['cik']

        key_words = ["healthcare", "financial services", "target return funds", "balanced funds", "mid cap funds"]
        # target_string = "8090 Services-Misc finance services, Health & Allied care Services, NEC balance fund"

        matched_words = [word for word in key_words if fuzzy_match(word, sic)]

        output_dict={}
        if len(matched_words)>0:
            output_dict['ticker']=str(ticker)
            output_dict['company_name']=company_name
            output_dict['cik']=cik
            output_dict['matches']=matched_words
            output_list.append(output_dict)

print(output_list)

# searching for the tickers with matches the Form-4

form_4_dict = {}


for match in output_list:
    ticker = f'{match["ticker"]}'
    if ticker not in form_4_dict.keys():
        # print('true')

        queryApi = QueryApi(api_key="438f72916c512e66d4320ebb0993972d141fb09f3ad14369c56e978bf6697432")
        query = {
        "query": { "query_string": {
            "query": 'formType:\"4\" AND filedAt:{2021-01-31 TO *} AND ticker:' + ticker
            } },
        "from": "0",
        "size": "100",
        "sort": [{ "filedAt": { "order": "desc" } }]
        }

        # AND ticker:\"\" TO *"

        filings_4 = queryApi.get_filings(query)
        
        filing_4 = filings_4['filings']
        
        for b in filing_4:
            comp_name = (b['companyName'])
            form_4_comp_name_list = []
            if comp_name not in form_4_comp_name_list:
                form_4_comp_name_list.append(comp_name)
                print(form_4_comp_name_list)
                form_4_dict[ticker]=form_4_comp_name_list
            # output_dict['form_4']=form_4_comp_name_list
            
            # output_list.append(output_dict)

    # if ticker in form_4_dict.keys():
        

[{'ticker': 'CPSS', 'company_name': 'CONSUMER PORTFOLIO SERVICES, INC.', 'cik': '889609', 'matches': ['financial services']}, {'ticker': 'AZZ', 'company_name': 'AZZ INC', 'cik': '8947', 'matches': ['financial services']}, {'ticker': 'DX', 'company_name': 'DYNEX CAPITAL INC', 'cik': '826675', 'matches': ['healthcare']}, {'ticker': 'CLB', 'company_name': 'Core Laboratories Inc. /DE/', 'cik': '1958086', 'matches': ['financial services']}, {'ticker': 'VVI', 'company_name': 'VIAD CORP', 'cik': '884219', 'matches': ['financial services']}, {'ticker': 'CPSS', 'company_name': 'CONSUMER PORTFOLIO SERVICES, INC.', 'cik': '889609', 'matches': ['financial services']}, {'ticker': 'CASS', 'company_name': 'CASS INFORMATION SYSTEMS INC', 'cik': '708781', 'matches': ['financial services']}, {'ticker': 'RYN', 'company_name': 'RAYONIER INC', 'cik': '52827', 'matches': ['healthcare']}, {'ticker': 'WHR', 'company_name': 'WHIRLPOOL CORP /DE/', 'cik': '106640', 'matches': ['balanced funds']}, {'ticker': 'CI'

In [211]:
print(ticker)

FMS


In [212]:
print(output_list)

[{'ticker': 'CPSS', 'company_name': 'CONSUMER PORTFOLIO SERVICES, INC.', 'cik': '889609', 'matches': ['financial services']}, {'ticker': 'AZZ', 'company_name': 'AZZ INC', 'cik': '8947', 'matches': ['financial services']}, {'ticker': 'DX', 'company_name': 'DYNEX CAPITAL INC', 'cik': '826675', 'matches': ['healthcare']}, {'ticker': 'CLB', 'company_name': 'Core Laboratories Inc. /DE/', 'cik': '1958086', 'matches': ['financial services']}, {'ticker': 'VVI', 'company_name': 'VIAD CORP', 'cik': '884219', 'matches': ['financial services']}, {'ticker': 'CPSS', 'company_name': 'CONSUMER PORTFOLIO SERVICES, INC.', 'cik': '889609', 'matches': ['financial services']}, {'ticker': 'CASS', 'company_name': 'CASS INFORMATION SYSTEMS INC', 'cik': '708781', 'matches': ['financial services']}, {'ticker': 'RYN', 'company_name': 'RAYONIER INC', 'cik': '52827', 'matches': ['healthcare']}, {'ticker': 'WHR', 'company_name': 'WHIRLPOOL CORP /DE/', 'cik': '106640', 'matches': ['balanced funds']}, {'ticker': 'CI'

In [213]:
# print(output_list)

for matched_dict in output_list:
    if matched_dict['ticker'] in form_4_dict.keys():
        ticker = matched_dict['ticker']
        form_4_comp_name_list = form_4_dict[ticker]
        matched_dict['form_4'] = form_4_comp_name_list

print(output_list)

[{'ticker': 'CPSS', 'company_name': 'CONSUMER PORTFOLIO SERVICES, INC.', 'cik': '889609', 'matches': ['financial services'], 'form_4': ['COSTCO WHOLESALE CORP /NEW']}, {'ticker': 'AZZ', 'company_name': 'AZZ INC', 'cik': '8947', 'matches': ['financial services'], 'form_4': ['COSTCO WHOLESALE CORP /NEW']}, {'ticker': 'DX', 'company_name': 'DYNEX CAPITAL INC', 'cik': '826675', 'matches': ['healthcare'], 'form_4': ['COSTCO WHOLESALE CORP /NEW']}, {'ticker': 'CLB', 'company_name': 'Core Laboratories Inc. /DE/', 'cik': '1958086', 'matches': ['financial services'], 'form_4': ['COSTCO WHOLESALE CORP /NEW']}, {'ticker': 'VVI', 'company_name': 'VIAD CORP', 'cik': '884219', 'matches': ['financial services'], 'form_4': ['COSTCO WHOLESALE CORP /NEW']}, {'ticker': 'CPSS', 'company_name': 'CONSUMER PORTFOLIO SERVICES, INC.', 'cik': '889609', 'matches': ['financial services'], 'form_4': ['COSTCO WHOLESALE CORP /NEW']}, {'ticker': 'CASS', 'company_name': 'CASS INFORMATION SYSTEMS INC', 'cik': '708781',

In [214]:
import pandas as pd 
df = pd.DataFrame(output_list, index=None)
df['matches'] = df['matches'].astype(str)
display(df)

,ticker,company_name,cik,matches,form_4
0,CPSS,"CONSUMER PORTFOLIO SERVICES, INC.",889609,['financial services'],[COSTCO WHOLESALE CORP /NEW]
1,AZZ,AZZ INC,8947,['financial services'],[COSTCO WHOLESALE CORP /NEW]
2,DX,DYNEX CAPITAL INC,826675,['healthcare'],[COSTCO WHOLESALE CORP /NEW]
3,CLB,Core Laboratories Inc. /DE/,1958086,['financial services'],[COSTCO WHOLESALE CORP /NEW]
4,VVI,VIAD CORP,884219,['financial services'],[COSTCO WHOLESALE CORP /NEW]
5,CPSS,"CONSUMER PORTFOLIO SERVICES, INC.",889609,['financial services'],[COSTCO WHOLESALE CORP /NEW]
6,CASS,CASS INFORMATION SYSTEMS INC,708781,['financial services'],[COSTCO WHOLESALE CORP /NEW]
7,RYN,RAYONIER INC,52827,['healthcare'],[COSTCO WHOLESALE CORP /NEW]
8,WHR,WHIRLPOOL CORP /DE/,106640,['balanced funds'],[COSTCO WHOLESALE CORP /NEW]
9,CI,Cigna Group,1739940,['financial services'],[COSTCO WHOLESALE CORP /NEW]
